# Cassini Example

In [1]:
import numpy as np

from builtins import input

import spiceypy as spice

In [2]:
# Load meta-kernel
spice.furnsh('cassini.tm')

## Time Conversion

Write a program that prompts the user for an input UTC time string, converts it to the following time systems and
output formats:
1. Ephemeris Time (ET) in seconds past J2000
2. Calendar Ephemeris Time
3. Spacecraft Clock Time

In [3]:
SCLKID = -82
# Prompt the user for an input UTC time string.
# Enter 2004 jun 11 19:32:00
utc_time = input('Enter time (UTC): ')

Enter time (UTC): 2004 jun 11 19:32:00


In [4]:
# Convert the input time string into ephemeris time expressed as 
# seconds past J2000 TDB. Display the result.
et_time = spice.str2et(utc_time)
print('Ephemeris time: ', et_time)

Ephemeris time:  140254384.1846252


In [5]:
# Convert ephemeris time into a calendar format. Display the
# result.
cal_time = spice.etcal(et_time)
print('Calendar format: ', cal_time)

Calendar format:  2004 JUN 11 19:33:04.184


In [6]:
# Convert ephemeris time into a spacecraft clock string. Display
# the result.
sclk_time = spice.sce2c(SCLKID, et_time)
print('Spacecraft clock time: ', sclk_time)

Spacecraft clock time:  197491442025.3071


## Obtaining Target States and Positions (getsta)
Prompt user for UTC time
1. The apparent state of Phoebe as seen from CASSINI in the J2000
frame, in kilometers and kilometers/second. This vector itself
is not of any particular interest, but it is a useful
intermediate quantity in some geometry calculations.
2. The apparent position of the Earth as seen from CASSINI in the
J2000 frame, in kilometers.
3. The one-way light time between CASSINI and the apparent
position of Earth, in seconds.
4. The apparent position of the Sun as seen from Phoebe in the
J2000 frame (J2000), in kilometers.
5. The actual (geometric) distance between the Sun and Phoebe, in
astronomical units.

In [7]:
# The apparent state of Phoebe as seen from CASSINI in the J2000 frame, in kilometers 
# and kilometers/second. This vector itself is not of any particular interest, but it 
# is a useful intermediate quantity in some geometry calculations.
[phoebe_state, phoebe_lt] = spice.spkezr('PHOEBE', et_time, 'J2000', 'LT+S', 'CASSINI')
print(phoebe_state)

[-1.19920929e+02  2.19413934e+03 -5.76389799e+01 -5.98023114e+00
 -2.11880531e+00 -2.94822134e-01]


In [8]:
# The apparent position of the Earth as seen from CASSINI in the J2000 frame, in kilometers.
# Earth NAIF ID: 399, Cassini NAIF ID: -82
[earth_pos, earth_lt] = spice.spkezp(399, et_time, 'J2000', 'LT+S', -82)
print(earth_pos)

[ 3.53019393e+08 -1.32818035e+09 -5.68134172e+08]


In [9]:
# The one-way light time between CASSINI and the apparent position of Earth, in seconds.
print(earth_lt)

4960.426912028222


In [10]:
# The apparent position of the Sun as seen from Phoebe in the J2000 frame (J2000), 
# in kilometers.
# Sun NAIF ID: 10, Phoebe NAIF ID: 609
[sun_pos, sun_lt] = spice.spkezp(10, et_time, 'J2000', 'LT+S', 609)
print(sun_pos)

[ 3.76551465e+08 -1.19049563e+09 -5.08438699e+08]


In [11]:
# The actual (geometric) distance between the Sun and Phoebe, in astronomical units.
sun_dist = spice.vnorm(sun_pos)
sun_dist_au = spice.convrt(sun_dist, 'KM', 'AU')
print(sun_dist_au)

9.012005475539208


## Spacecraft Orientation and Reference Frames

Write a program that prompts the user for an input time string, computes and displays the following at the epoch of interest:
1. The apparent state of Phoebe as seen from CASSINI in the IAU_PHOEBE body-fixed frame. This vector itself is not of any particular interest, but it is a useful intermediate quantity in some geometry calculations.
2. The angular separation between the apparent position of Earth as seen from CASSINI and the nominal boresight of the CASSINI high gain antenna (HGA). The HGA boresight direction is provided by the kernel variable TKFRAME_-82101_BORESIGHT, which is defined in the Cassini frame kernel cited above in the section "Kernels Used." In this kernel, the HGA boresight vector is expressed relative to the CASSINI_HGA reference frame.


In [12]:
# The apparent state of Phoebe as seen from CASSINI in the IAU_PHOEBE body-fixed frame. 
# This vector itself is not of any particular interest, but it is a useful intermediate 
# quantity in some geometry calculations.
[phoebe_state_iau, phoebe_lt_iau] = spice.spkezr('PHOEBE', et_time, 'IAU_PHOEBE', 'LT+S', 'CASSINI')
print(phoebe_state_iau)

[-1982.63976162  -934.53047112  -166.56259513     3.97083213
    -3.81249566    -2.37166299]


In [13]:
# Apparent position of Earth as seen from CASSINI in CASSINI_HGA frame
[earth_pos, earth_ltime] = spice.spkpos('EARTH', et_time, 'CASSINI_HGA', 'LT+S', 'CASSINI')
boresight_pos = [0.0, 0.0, 1.0]
print(earth_pos)

[4.97510258e+08 1.32327091e+09 4.61410682e+08]


In [14]:
# Angular separation
earth_bore_sep = spice.vsep(earth_pos, boresight_pos)
print(earth_bore_sep)

1.2553132026561336


In [15]:
# Convert from radians to degrees
earth_bore_deg = spice.convrt(earth_bore_sep, 'RADIANS', 'DEGREES')
print(earth_bore_deg)

71.92414847924705


## Computing Sub-s/c and Sub-solar Points on an Ellipsoid and a DSK

1. The apparent sub-observer point of CASSINI on Phoebe, in the body fixed frame IAU_PHOEBE, in kilometers.
2. The apparent sub-solar point on Phoebe, as seen from CASSINI in the body fixed frame IAU_PHOEBE, in kilometers.

In [16]:
# The apparent sub-observer point of CASSINI on Phoebe, in the body fixed frame IAU_PHOEBE, 
# in kilometers. Using near point/ellipsoid.
# See: https://naif.jpl.nasa.gov/pub/naif/toolkit_docs/C/cspice/subpnt_c.html
[subobs_point, subobs_epoch, subobs_vec] = spice.subpnt('near point/ellipsoid', 'PHOEBE', 
                                                        et_time,'IAU_PHOEBE', 'LT+S', 
                                                        'CASSINI')
print(subobs_point)

[104.49789074  45.26884577   7.38331473]


In [17]:
# The apparent sub-solar point on Phoebe, as seen from CASSINI in the body fixed frame 
# IAU_PHOEBE, in kilometers. Using near point/ellipsoid.
# See: https://naif.jpl.nasa.gov/pub/naif/toolkit_docs/C/cspice/subslr_c.html
[subsol_point, subsol_epoch, subsol_vec] = spice.subslr('near point/ellipsoid', 'PHOEBE', 
                                                        et_time,'IAU_PHOEBE', 'LT+S', 
                                                        'CASSINI')
print(subsol_point)

[ 78.68071625  76.8786516  -21.88456729]


In [18]:
# The apparent sub-observer point of CASSINI on Phoebe, in the body fixed frame IAU_PHOEBE, 
# in kilometers. Using NADIR/DSK/Unprioritized.
[subobs_point_dsk, subobs_epoch_dsk, subobs_vec_dsk] = spice.subpnt('NADIR/DSK/Unprioritized', 
                                                'PHOEBE', et_time,'IAU_PHOEBE', 'LT+S', 
                                                'CASSINI')
print(subobs_point_dsk)

[95.37257468 40.94817689  6.6099027 ]


In [19]:
# The apparent sub-solar point on Phoebe, as seen from CASSINI in the body fixed frame 
# IAU_PHOEBE, in kilometers. Using NADIR/DSK/Unprioritized.
[subsol_point_dsk, subsol_epoch_dsk, subsol_vec_dsk] = spice.subslr('NADIR/DSK/Unprioritized', 
                                                'PHOEBE', et_time,'IAU_PHOEBE', 'LT+S', 
                                                'CASSINI')
print(subsol_point_dsk)

[ 79.11113709  77.33831624 -22.02817575]
